#### DivideFromHere

# Battles

### Battle Design

### Step 2: Define the Different Versions of Kane and Abel

In [ ]:
import chess
import time
import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, SVG
import torch
import torch.nn as nn
import torch.optim as optim

# Define the different versions of Kane and Abel
kane_versions = {
    'minimax': KaneMinimaxDeterministic,
    'alpha_beta': KaneAlphaBetaDeterministic,
    'minimax_randomized': KaneMinimaxRandomization,
    'alpha_beta_randomized': KaneAlphaBetaRandomization
}

abel_versions = {
    'simple_nn': AbelSimpleNN,
    'dqn': AbelDQN,
    'simple_nn_derandomized': AbelSimpleNNDerandomized,
    'dqn_derandomized': AbelDQNDerandomized
}

# Define the battle parameters
num_games = 100
max_moves = 55
max_runtime = 600


## Step 3: Define Play Functions for Kane and Abel
### Chess Environment Setup

In [ ]:
class ChessEnv:
    def reset(self):
        self.board = chess.Board()
        return self.board

    def step(self, action):
        self.board.push(action)
        reward = self.evaluate_board(self.board)
        done = self.board.is_game_over()
        next_state = self.board
        return next_state, reward, done

    def legal_moves(self):
        return list(self.board.legal_moves)

    def simulate_move(self, move):
        board_copy = self.board.copy()
        board_copy.push(move)
        return board_copy

    def evaluate_board(self, board):
        return sum(1 if piece.color == chess.WHITE else -1 for piece in board.piece_map().values())


### Play Functions

In [ ]:
def play_game_kane(kane, env, depth=3, max_moves=55, max_runtime=600):
    kane_moves = []
    start_time = time.time()

    for move_num in range(max_moves):
        if env.board.is_game_over() or (time.time() - start_time) > max_runtime:
            break
        
        # Kane's move
        kane_move = kane.find_best_move(depth)
        env.step(kane_move)
        kane_moves.append(kane_move)
        
        if env.board.is_game_over():
            break

    return kane_moves

def play_game_abel(abel, env, max_moves=55, max_runtime=600):
    abel_moves = []
    start_time = time.time()

    for move_num in range(max_moves):
        if env.board.is_game_over() or (time.time() - start_time) > max_runtime:
            break
        
        # Abel's move
        abel_move = abel.find_best_move()
        env.step(abel_move)
        abel_moves.append(abel_move)
        
        if env.board.is_game_over():
            break

    return abel_moves


## Step 4: Simulate Battles

In [ ]:
def simulate_battle(kane_class, abel_class, kane_seed=42, abel_seed=42, num_games=100, max_moves=55, max_runtime=600):
    env = ChessEnv()
    kane = kane_class(env.board, kane_seed)
    abel = abel_class(env.board, abel_seed)
    
    kane_results = []
    abel_results = []
    
    for _ in range(num_games):
        env.reset()
        kane_moves = play_game_kane(kane, env, depth=3, max_moves=max_moves, max_runtime=max_runtime)
        abel_moves = play_game_abel(abel, env, max_moves=max_moves, max_runtime=max_runtime)
        
        # Collect results
        kane_results.append(kane_moves)
        abel_results.append(abel_moves)
    
    return kane_results, abel_results

# Example usage
kane_results, abel_results = simulate_battle(KaneMinimaxDeterministic, AbelSimpleNN)


## Step 5: Aggregate and Analyze Performance Metrics

In [ ]:
def aggregate_battle_metrics(kane_results, abel_results):
    def calculate_metrics(moves):
        material_counts = []
        mobility_counts = []
        piece_square_scores = []
        center_control_counts = []
        
        for move in moves:
            board = chess.Board()
            for m in move:
                board.push(m)
            
            material_count = sum(1 if piece.color == chess.WHITE else -1 for piece in board.piece_map().values())
            mobility_count = len(list(board.legal_moves))
            piece_square_score = sum(1 if piece.color == chess.WHITE else -1 for piece in board.piece_map().values())
            center_control_count = sum(1 if square in [chess.D4, chess.E4, chess.D5, chess.E5] else 0 for square, piece in board.piece_map().items())
            
            material_counts.append(material_count)
            mobility_counts.append(mobility_count)
            piece_square_scores.append(piece_square_score)
            center_control_counts.append(center_control_count)
        
        return {
            'Material Count': material_counts,
            'Mobility Count': mobility_counts,
            'Piece-Square Score': piece_square_scores,
            'Center Control Count': center_control_counts
        }
    
    kane_metrics = calculate_metrics(kane_results)
    abel_metrics = calculate_metrics(abel_results)
    
    return kane_metrics, abel_metrics

# Aggregate the results
kane_metrics, abel_metrics = aggregate_battle_metrics(kane_results, abel_results)

# Display the aggregated metrics
print("Kane Metrics:\n", kane_metrics)
print("Abel Metrics:\n", abel_metrics)


## Step 6: Integrate KAN for Training
### KAN Model Definition

In [ ]:
class KANModel(nn.Module):
    def __init__(self):
        super(KANModel, self).__init__()
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_kan_model(data, labels, num_epochs=100):
    model = KANModel()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        inputs = torch.tensor(data, dtype=torch.float32)
        targets = torch.tensor(labels, dtype=torch.float32)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}/{num_epochs}, Loss: {loss.item()}')

    return model

def prepare_kan_data(kane_metrics, abel_metrics):
    data = np.concatenate((list(kane_metrics.values()), list(abel_metrics.values())), axis=1)
    labels = np.zeros((len(data[0]), 1))
    return data.T, labels

# Prepare the data for KAN model
data, labels = prepare_kan_data(kane_metrics, abel_metrics)
kan_model = train_kan_model(data, labels)


## Step 7: Visualization
### Plot Battle Results

In [ ]:
def plot_battle_results(kane_metrics, abel_metrics):
    metrics = list(kane_metrics.keys())
    
    fig, axs = plt.subplots(len(metrics), 1, figsize=(10, 20))
    
    for i, metric in enumerate(metrics):
        axs[i].plot(kane_metrics[metric], label='Kane')
        axs[i].plot(abel_metrics[metric], label='Abel')
        axs[i].set_title(metric)
        axs[i].legend()
    
    plt.tight_layout()
    plt.show()

# Plot the battle results
plot_battle_results(kane_metrics, abel_metrics)


### Plot Equivalence Curves

In [ ]:
def plot_equivalence_curves(kane_metrics, abel_metrics):
    metrics = list(kane_metrics.keys())
    x = range(len(kane_metrics[metrics[0]]))

    fig, ax = plt.subplots(figsize=(14, 7))

    for metric in metrics:
        kane_mean = np.mean(kane_metrics[metric])
        kane_std = np.std(kane_metrics[metric])
        abel_mean = np.mean(abel_metrics[metric])
        abel_std = np.std(abel_metrics[metric])

        ax.errorbar(x, [kane_mean] * len(x), yerr=[kane_std] * len(x), fmt='o-', label=f'Kane {metric}', capsize=5)
        ax.errorbar(x, [abel_mean] * len(x), yerr=[abel_std] * len(x), fmt='o-', label=f'Abel {metric}', capsize=5)

    ax.set_title('Equivalence Curves for Kane and Abel')
    ax.set_xlabel('Games')
    ax.set_ylabel('Values')
    ax.legend()
    plt.tight_layout()
    plt.show()

# Plot the equivalence curves
plot_equivalence_curves(kane_metrics, abel_metrics)


## Step 8: Perform Statistical Tests
### Statistical Tests and Analysis

In [ ]:
from scipy.stats import ttest_ind, f_oneway

def perform_statistical_tests(kane_metrics, abel_metrics):
    results = {}
    for metric in kane_metrics.keys():
        t_stat, p_value_t = ttest_ind(kane_metrics[metric], abel_metrics[metric], equal_var=False)
        f_stat, p_value_f = f_oneway(kane_metrics[metric], abel_metrics[metric])
        results[metric] = {
            't_stat': t_stat,
            'p_value_t': p_value_t,
            'f_stat': f_stat,
            'p_value_f': p_value_f
        }
    return results

# Perform statistical tests
statistical_results = perform_statistical_tests(kane_metrics, abel_metrics)

# Display the results
for metric, result in statistical_results.items():
    print(f"{metric}: t-statistic = {result['t_stat']}, p-value (t-test) = {result['p_value_t']}")
    print(f"{metric}: f-statistic = {result['f_stat']}, p-value (F-test) = {result['p_value_f']}\n")


### Plot Statistical Analysis Results

In [ ]:
def plot_statistical_analysis(statistical_results):
    metrics = list(statistical_results.keys())
    t_stats = [result['t_stat'] for result in statistical_results.values()]
    p_values_t = [result['p_value_t'] for result in statistical_results.values()]

    fig, axs = plt.subplots(2, 1, figsize=(14, 10))

    # Plot t-statistics
    axs[0].bar(metrics, t_stats, color='blue')
    axs[0].set_xticks(metrics)
    axs[0].set_xticklabels(metrics, rotation=45, ha='right')
    axs[0].set_ylabel('t-statistic')
    axs[0].set_title('t-statistic of Each Metric')

    # Plot p-values (t-test)
    axs[1].bar(metrics, p_values_t, color='green')
    axs[1].set_xticks(metrics)
    axs[1].set_xticklabels(metrics, rotation=45, ha='right')
    axs[1].axhline(y=0.05, color='r', linestyle='--')
    axs[1].set_ylabel('p-value (t-test)')
    axs[1].set_title('p-value (t-test) of Each Metric')

    plt.tight_layout()
    plt.show()

# Plot statistical analysis results
plot_statistical_analysis(statistical_results)


## Step 10: Integrate KAN for Interpretation
### Define the KAN Model

In [ ]:
class KANModel(nn.Module):
    def __init__(self):
        super(KANModel, self).__init__()
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_kan_model(data, labels, num_epochs=100):
    model = KANModel()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    losses = []
    for epoch in range(num_epochs):
        inputs = torch.tensor(data, dtype=torch.float32)
        targets = torch.tensor(labels, dtype=torch.float32)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        if epoch % 10 == 0:
            print(f'Epoch {epoch}/{num_epochs}, Loss: {loss.item()}')

    return model, losses

def prepare_kan_data(kane_metrics, abel_metrics):
    data = np.concatenate((list(kane_metrics.values()), list(abel_metrics.values())), axis=1)
    labels = np.zeros((len(data[0]), 1))
    return data.T, labels

# Prepare the data for KAN model
data, labels = prepare_kan_data(kane_metrics, abel_metrics)
kan_model, losses = train_kan_model(data, labels)


## Step 11: Visualize KAN Model and Results
### Visualize KAN Model Weights and Biases

In [ ]:
def visualize_kan_weights_biases(kan_model):
    for name, param in kan_model.named_parameters():
        if param.requires_grad:
            print(f"{name}: {param.data}")

visualize_kan_weights_biases(kan_model)

import matplotlib.pyplot as plt

def plot_kan_training_loss(losses):
    plt.figure(figsize=(10, 5))
    plt.plot(losses, label='Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('KAN Model Training Loss')
    plt.legend()
    plt.show()

plot_kan_training_loss(losses)


## Step 12: Extract and Interpret the Equivalence Formula
### Extract Symbolic Formula

In [ ]:
def extract_equivalence_formula(kan_model):
    weights = kan_model.fc1.weight.data.numpy()
    biases = kan_model.fc1.bias.data.numpy()
    formula = " + ".join([f"{weights[0][i]}*x{i}" for i in range(weights.shape[1])])
    formula += f" + {biases[0]}"
    return formula

equivalence_formula = extract_equivalence_formula(kan_model)
print(f"Equivalence Formula: {equivalence_formula}")


## Step 13: Generate Equivalence Curves
### Generate and Plot Equivalence Curves

In [ ]:
def generate_equivalence_curve(kan_model, data_loader):
    equivalence_scores = []
    for inputs, _ in data_loader:
        inputs = inputs.float()
        outputs = kan_model(inputs).detach().numpy()
        equivalence_scores.extend(outputs)
    return equivalence_scores

equivalence_scores = generate_equivalence_curve(kan_model, kan_data_loader)

def plot_equivalence_curve(equivalence_scores):
    plt.figure(figsize=(10, 5))
    plt.plot(equivalence_scores, label='Equivalence Scores')
    plt.xlabel('Samples')
    plt.ylabel('Equivalence Score')
    plt.title('Equivalence Curve')
    plt.legend()
    plt.show()

plot_equivalence_curve(equivalence_scores)


## Step 14: Conclusion and Insights
### Summarize Results and Insights

In [ ]:
def summarize_results(equivalence_score, kan_evaluation_loss, equivalence_formula):
    print(f"Equivalence Score: {equivalence_score}")
    print(f"KAN Model Evaluation Loss: {kan_evaluation_loss}")
    print(f"Extracted Equivalence Formula: {equivalence_formula}")
    print("The equivalence score and KAN model insights suggest that the deterministic and probabilistic versions of the models are closely related.")
    print("The extracted formula provides a mathematical representation of this relationship, further validating the potential equivalence between deterministic and probabilistic AI models in the context of chess.")

summarize_results(equivalence_score, kan_evaluation_loss, equivalence_formula)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6f51e1a-d40a-494a-8398-36807e7a81cb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>